# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [1]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [2]:
### General parameters
max_steps = 5e10 ##5e5 # Set maximum number of steps to run environment.
run_path = "ppo" # The sub-directory name for model and summary statistics
load_model = False # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 10000 # Frequency at which to save training statistics.
save_freq = 50000 # Frequency at which to save model.
env_name = "ObstacleCurriculum" # Name of the training environment file.
curriculum_file = None

### Algorithm-specific parameters for tuning
gamma = 0.99 #0.99 # Reward discount rate.
lambd = 0.95 ##0.95 # Lambda parameter for GAE.
time_horizon = 2048 ##2048 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 ##1e-3 # Strength of entropy regularization
num_epoch = 5 ##5 # Number of gradient descent steps per batch of experiences.
num_layers = 2 ##2 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.2 ##0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 4096 ##2048 # How large the experience buffer should be before gradient descent.
learning_rate = 3e-4 # Model learning rate.
hidden_units = 128 ##64 # Number of units in hidden layer.
batch_size = 32 ##64 # How many experiences per gradient descent update step.
normalize = False

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [3]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'ObstacleCurriculumAcademy' started successfully!


Unity Academy name: ObstacleCurriculumAcademy
        Number of brains: 1
        Reset Parameters :
		
Unity brain name: ObstacleCurriculumBrain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 9
        Action space type: discrete
        Action space size (per agent): 8
        Memory space size (per agent): 0
        Action descriptions: , , , , , , , 


### Train the Agent(s)

In [4]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Step: 10000. Mean Reward: -2.957192985105239. Std of Reward: 2.7339894146717194.
Step: 20000. Mean Reward: -3.9787500028749605. Std of Reward: 2.7575518049844994.
Step: 30000. Mean Reward: -6.531562504687429. Std of Reward: 3.4786931550326825.
Step: 40000. Mean Reward: -3.93483333769996. Std of Reward: 2.567898239007162.
Step: 50000. Mean Reward: -8.593939400363682. Std of Reward: 7.53625303170814.
Saved Model
Step: 60000. Mean Reward: -6.249313729176419. Std of Reward: 5.031749362987308.
Step: 70000. Mean Reward: -4.995104170687437. Std of Reward: 3.0578013547018372.
Step: 80000. Mean Reward: -4.8710000035999474. Std of Reward: 3.744505768114265.
Step: 90000. Mean Reward: -4.5079166724999435. Std of Reward: 3.521069278291441.
Step: 100000. Mean Reward: -8.67944444677764. Std of Reward: 5.394263469609611.
Saved Model
Step: 110000. Mean Reward: -5.67862745458817. Std of Reward: 4.433387153357006.
Step: 120000. Mean Reward: -3.3440384656538122. Std of Reward: 2.4129653435596197.
Step: 13

Step: 980000. Mean Reward: 0.6078288267459458. Std of Reward: 0.3716438008268471.
Step: 990000. Mean Reward: 0.6170599233764044. Std of Reward: 0.32659858669557157.
Step: 1000000. Mean Reward: 0.6224063653314605. Std of Reward: 0.3093349943137739.
Saved Model
Step: 1010000. Mean Reward: 0.6091569750290703. Std of Reward: 0.5635198618306414.
Step: 1020000. Mean Reward: 0.5071478855352127. Std of Reward: 1.0916589895481799.
Step: 1030000. Mean Reward: 0.6263394664969327. Std of Reward: 0.38153252886593536.
Step: 1040000. Mean Reward: 0.6023108364846634. Std of Reward: 0.8329111673780921.
Step: 1050000. Mean Reward: 0.6702162140594594. Std of Reward: 0.2604313492314153.
Saved Model
Step: 1060000. Mean Reward: 0.6421228051894735. Std of Reward: 0.339257612262162.
Step: 1070000. Mean Reward: 0.6014065487283243. Std of Reward: 0.8111065370373916.
Step: 1080000. Mean Reward: 0.590612242520408. Std of Reward: 0.45737180450766457.
Step: 1090000. Mean Reward: 0.6209867308059701. Std of Reward: 0

Step: 1940000. Mean Reward: 0.36632849947824697. Std of Reward: 2.8421080884528647.
Step: 1950000. Mean Reward: 0.6301472838093023. Std of Reward: 0.44971959834179953.
Saved Model
Step: 1960000. Mean Reward: 0.576996924861751. Std of Reward: 0.5353023901403439.
Step: 1970000. Mean Reward: 0.45178727782893885. Std of Reward: 2.28524138385734.
Step: 1980000. Mean Reward: 0.5609860354031422. Std of Reward: 0.9592277875829726.
Step: 1990000. Mean Reward: 0.4413647313840452. Std of Reward: 3.061297673536694.
Step: 2000000. Mean Reward: 0.4959959321341428. Std of Reward: 1.7978646029237688.
Saved Model
Step: 2010000. Mean Reward: 0.2867787086890593. Std of Reward: 3.0087517360991645.
Step: 2020000. Mean Reward: 0.5014320955481433. Std of Reward: 2.0575311642464684.
Step: 2030000. Mean Reward: -0.831994050875015. Std of Reward: 7.961970076410234.
Step: 2040000. Mean Reward: 0.22680685059811892. Std of Reward: 3.705026279742367.
Step: 2050000. Mean Reward: -0.007067904185221856. Std of Reward:

error: unpack requires a bytes object of length 4

### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [5]:
export_graph(model_path, env_name)

INFO:tensorflow:Restoring parameters from ./models/ppo\model-2750000.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo\model-2750000.cptk


INFO:tensorflow:Froze 4 variables.


INFO:tensorflow:Froze 4 variables.


Converted 4 variables to const ops.
